# Coder Agent Workspace

Drive the lightweight `CoderAgent` to spin up focused Python projects with your local tool-calling LLM.

## Imports & configuration

Set the endpoint, model, and folders once, then reuse the agent across briefs.

In [4]:
from core.agent import CoderAgent
from core.config import LLMConfig
from pathlib import Path
import json
BASE_URL = "http://localhost:1234/v1"
MODEL = "qwen/qwen3-coder-30b"
PROJECT_ROOT = "generated_projects"
LOG_DIR = "logs"
print(f"Base URL: {BASE_URL}")
print(f"Model: {MODEL}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Log dir: {LOG_DIR}")


Base URL: http://localhost:1234/v1
Model: qwen/qwen3-coder-30b
Project root: generated_projects
Log dir: logs


## Instantiate the coder agent

This keeps the workspace isolated and exposes filesystem tools to the LLM.

In [5]:
config = LLMConfig.load()
config.base_url = BASE_URL
config.model = MODEL
config.log_dir = LOG_DIR

coder = CoderAgent(
    config=config,
    project_root=PROJECT_ROOT,
    verbose=False,
)

print("Coder agent ready.")


Coder agent ready.


## Set the project brief

Use this cell to describe the project you want the coder agent to build. It currently defaults to a React calculator demo so you can validate the workflow quickly.


In [ ]:
project_brief = """Build a simple React calculator web app.

Requirements:
- Use React 18 via CDN (no bundler) with a single HTML host page.
- Implement a calculator UI with digits, basic operators, equals, AC, and delete actions.
- Style it so it looks modern on desktop and mobile.
- Include a README with steps to run a simple static server.
"""
project_name = "react_calculator"  # Set to None to auto-generate from the brief
max_iterations = 6

print("Current project name:", project_name)
print(project_brief)


## Build the project

The agent loops through chat + tool calls until it finishes or hits the iteration cap.

In [ ]:
coder.reset_conversation()
summary = coder.build_project(
    project_description=project_brief,
    project_name=project_name,
    max_iterations=max_iterations,
)
print("Build summary:", summary)

## Inspect generated files

List everything inside the active workspace.

## Helper: run new briefs quickly

Use this function in ad-hoc cells to kick off additional builds without reconfiguring anything.

In [6]:
def run_brief(description: str, *, name: str | None = None, iterations: int = 6):
    coder.reset_conversation()
    result = coder.build_project(description, project_name=name, max_iterations=iterations)
    print(result)
    return result

# Example usage:
# run_brief("Generate a simple stopwatch CLI with tests", name="stopwatch")

## Runner tool test

Ask the agent to use the `run_shell` tool to create a folder and file inside the workspace, then show the result.



In [7]:
runner_test_brief = """Use the run_shell tool to:
- create a directory named 'demo_shell'
- create an empty file 'demo_shell/ok.txt'
- list files to confirm the new entries
If a single shell invocation is more convenient, you may chain commands like: mkdir -p demo_shell && touch demo_shell/ok.txt
Finally, respond with a short summary of what was created.
"""

# Execute the brief
run_brief(runner_test_brief, name="runner_tool_demo", iterations=4)


I've created a new directory named 'demo_shell' and an empty file 'ok.txt' inside it. The file listing confirms the new entries.

Here's a summary of what was created:
- A directory called 'demo_shell'
- An empty file named 'ok.txt' inside the demo_shell directory

The shell commands successfully created the directory structure and verified the file existence.


"I've created a new directory named 'demo_shell' and an empty file 'ok.txt' inside it. The file listing confirms the new entries.\n\nHere's a summary of what was created:\n- A directory called 'demo_shell'\n- An empty file named 'ok.txt' inside the demo_shell directory\n\nThe shell commands successfully created the directory structure and verified the file existence."